# Transformação de tipo de arquivo

> Visualizar pastas da conta de armazenamento

In [0]:
dbutils.fs.ls('abfss://livros@recomendador.dfs.core.windows.net/')

In [0]:
dbutils.fs.ls('abfss://livros@recomendador.dfs.core.windows.net/inbound')

## De csv para delta

> Ler arquivos

> IMPORTATE

> Os parâmetros _quote_ e _escape_ são essenciais para a delimitação dos valores de cada linha corresponderem às colunas.
> Nada mais são que, respectivamente, indicar que o texto entre aspas, mesmo que possua o delimitador, é um único valor não podendo ser separado; e especificar que dentro do texto com aspas podem haver outras aspas (no caso dos _datasets_ utilizados).

In [0]:
df_goodreads = spark.read.options(delimiter=",", header=True, quote='"', escape='"', multiline=True).csv('abfss://livros@recomendador.dfs.core.windows.net/inbound/goodreads_100k_books.csv', inferSchema=True)
df_kindle = spark.read.options(delimiter=",", header=True, quote='"', escape='"', multiline=True).csv('abfss://livros@recomendador.dfs.core.windows.net/inbound/kindle_data_v2.csv', inferSchema=True)

> Visualizar dados

In [0]:
df_kindle.toPandas().head(2)

In [0]:
df_goodreads.toPandas().head(2)

> Visualizar se há dados duplicados

In [0]:
df_kindle.groupBy(df_kindle.asin) \
        .count() \
        .orderBy('count', ascending=False) \
        .show(10)

In [0]:
df_kindle.groupBy(df_kindle.title) \
        .count() \
        .orderBy('count', ascending=False) \
        .show(10)

In [0]:
df_goodreads.groupBy(df_goodreads.isbn) \
        .count() \
        .orderBy('count', ascending=False) \
        .show(10)

In [0]:
df_goodreads.groupBy(df_goodreads.title) \
        .count() \
        .orderBy('count', ascending=False) \
        .show(10)

### Remoção dos dados duplicados

> Dados do _Kindle_

In [0]:
df_kindle.select(df_kindle.publishedDate, df_kindle.title).where(df_kindle.title == 'Organic Chemistry').orderBy(df_kindle.publishedDate.desc()).show(truncate=False)

In [0]:
df_kindle = df_kindle.orderBy(df_kindle.publishedDate.desc())
df_kindle = df_kindle.dropDuplicates(['title'])

> Dados do _GoodReads_

In [0]:
df_goodreads.select(df_goodreads.reviews, df_goodreads.rating, df_goodreads.totalratings, df_goodreads.title) \
            .where(df_goodreads.title == 'Love in the Mask') \
            .orderBy(df_goodreads.totalratings.desc()) \
            .show(truncate=False)

In [0]:
df_goodreads = df_goodreads.orderBy(df_goodreads.totalratings.desc()) \
                            .orderBy(df_goodreads.rating.desc()) \
                            .orderBy(df_goodreads.reviews.desc()) \
                            .dropDuplicates(['title'])

> Alterar formato dos arquivos de saída

In [0]:
df_goodreads.write.format('delta').options(header=True).mode('overwrite').save('abfss://livros@recomendador.dfs.core.windows.net/bronze/goodreads')
df_kindle.write.format('delta').options(header=True).mode('overwrite').save('abfss://livros@recomendador.dfs.core.windows.net/bronze/kindle')

## Exclusão de variáveis

In [0]:
del df_goodreads, df_kindle